In [1]:
print("hello")

hello


In [ ]:
! pip install python-dotenv
! pip install langchain langgraph langsmith
! pipinstall langchain-groq


In [4]:
from dotenv import load_dotenv
import os

In [ ]:
#create a .env file with the following content:
# groq_api_key=your_groq_api_key_here
# langsmith = your_langsmith_api_key_here

# Load variables from .env file into environment
load_dotenv()

In [14]:
# Load variables from .env file into environment
load_dotenv()

# Access them using os.getenv()
groq_api_key = os.getenv("groq_api_key")
langsmith = os.getenv("langsmith_api_key")

# langsmith

In [13]:
os.environ["LANGCHAIN_API_KEY"] = langsmith
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="CourseLanggraph"

In [15]:
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
# print(llm)

In [16]:

from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages

In [17]:
class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    user_input: str
    llm_response: str
    result: str

In [18]:
def call_llm(state: State) -> State:
    prompt = f"Classify this as a question or statement: {state['user_input']}"
    response = llm.invoke(prompt)
    return {"llm_response": response.content}

In [19]:
def classify(state: State) -> State:
    if "question" in state["llm_response"].lower():
        return {"result": "I'll try to answer that."}
    else:
        return {"result": "That's a statement."}

In [20]:
graph = StateGraph(State)
graph.add_node("llm", call_llm)
graph.add_node("classify", classify)
graph.set_entry_point("llm")
graph.add_edge("llm", "classify")
graph.set_finish_point("classify")

compiled=graph.compile()

In [22]:
user_input = input("You: ")
result = compiled.invoke({"user_input": user_input})
print("Assistant:", result["result"])

with open("langraph_graphs/SimpleLangraph.png", "wb") as f:
    f.write(compiled.get_graph().draw_mermaid_png())

Assistant: I'll try to answer that.
